In [89]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import theano

import os, sys, inspect
cmd_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile( inspect.currentframe() ))[0]))
if cmd_folder not in sys.path:
    sys.path.insert(0, cmd_folder + '/carl')

In [90]:
theta = [[1.0, -0.5, -0.5], 
         [1.0, -0.5, 0.5], 
         [1.0, -0.35714285714285715, -0.071428571428571452], 
         [1.0, -0.35714285714285715, 0.21428571428571419], 
         [1.0, -0.2142857142857143, -0.2142857142857143], 
         [1.0, -0.2142857142857143, 0.071428571428571397], 
         [1.0, -0.2142857142857143, 0.3571428571428571], 
         [1.0, -0.071428571428571452, -0.35714285714285715], 
         [1.0, -0.071428571428571452, -0.071428571428571452], 
         [1.0, -0.071428571428571452, 0.21428571428571419], 
         [1.0, 0.21428571428571419, -0.2142857142857143], 
         [1.0, 0.21428571428571419, 0.071428571428571397], 
         [1.0, 0.3571428571428571, 0.5], 
         [1.0, 0.5, -0.071428571428571452], 
         [1.0, 0.5, 0.21428571428571419],
         [1.0, -0.5, -0.35714285714285715], 
         [1.0, -0.5, -0.071428571428571452], 
         [1.0, -0.5, 0.3571428571428571], 
         [1.0, -0.2142857142857143, -0.071428571428571452], 
         [1.0, -0.2142857142857143, 0.21428571428571419], 
         [1.0, -0.2142857142857143, 0.5], 
         [1.0, -0.071428571428571452, -0.5], 
         [1.0, -0.071428571428571452, -0.2142857142857143], 
         [1.0, -0.071428571428571452, 0.071428571428571397], 
         [1.0, 0.071428571428571397, -0.071428571428571452], 
         [1.0, 0.071428571428571397, 0.3571428571428571], 
         [1.0, 0.21428571428571419, -0.35714285714285715], 
         [1.0, 0.21428571428571419, 0.21428571428571419], 
         [1.0, 0.5, -0.2142857142857143], 
         [1.0, 0.5, 0.5]]

theta = np.array(theta)

theta_0 = theta[:15]
theta_1 = theta[15:]

F1 = [1.0, 0.0, 0.0]
F0 = theta_0[0]

feature_names = ["minDelR_jZ", "DelPhi_Hjj", "DelEta_Hjj", "DelPhi_jj", "DelR_jj", 
                 "DelY_jj", "Mjj", "DelPt_jj", "DelPt_ZZ", "pT_Hjj", "Mjets", "Zeppetaj3", 
                 "ZeppetaZZ", "njets","Z1_E","Z1_pt","Z1_eta","Z1_phi","Z1_m","Z2_E","Z2_pt",
                 "Z2_eta","Z2_phi","Z2_m",  "higgs_E","higgs_pt","higgs_eta","higgs_phi","higgs_m", 
                 "jet1_E","jet1_eta","jet1_y","jet1_phi","jet1_pt","jet1_m","jet2_E", "jet2_eta","jet2_y", 
                 "jet2_phi","jet2_pt","jet2_m","jet3_E","jet3_eta","jet3_y" ,"jet3_phi","jet3_pt","jet3_m"]

subset = [33, 3] #DelPhi_jj, jet1_pt
observed = 'F1'
data_F0 = 0

c_min = [-0.3557, -0.34467]
#c_min = [-0.1,-0.1]
c_max = [0.2646, 0.34467]
npoints = 15

n_samples = 40000
test_samples = 15000

# subset = list(range(len(feature_names)))

In [91]:
X_observed = np.loadtxt("/afs/cern.ch/work/j/jpavezse/private/data/new_data/data_%s.dat" 
                        % observed)[:test_samples, subset]
X_observed.shape

# ideally, X_observed should be an independent sample not used for training

(15000, 2)

In [92]:
data_dir = "/afs/cern.ch/work/j/jpavezse/private/data/new_data"
# Mixture Model
class Component:
    def __init__(self, base):
        self.data = np.loadtxt("%s/data_%d.dat" % (data_dir,base))[:, subset]
    def rvs(self, n_samples, random_state=1234):
        n_samples = n_samples if n_samples < len(self.data) else len(self.data)
        return self.data[:n_samples]
    
class Mixture:
    def __init__(self, basis, couplings, cross_sections):
        self.basis = basis
        self.couplings = couplings
        self.cross_sections = cross_sections
        self.components = []
        for base in self.basis:
            self.components.append(Component(base))
    
    def compute_weights(self,**kwargs):
        couplings = np.array(map(lambda x: x.eval(),self.couplings))
        cross_sections = np.array(map(lambda x: x.eval(), self.cross_sections))
        cs = np.multiply(couplings, cross_sections)
        return cs / cs.sum()        


In [93]:
# Reading indexes, couplings and cross_sections
all_indexes = np.loadtxt(
    'doubleindexes_{0:.2f}_{1:.2f}_{2:.2f}_{3:.2f}_{4}.dat'.format(
        c_min[0], c_min[1], c_max[0], c_max[1], npoints))
all_indexes = np.array([[int(x) for x in rows]
                        for rows in all_indexes])
all_couplings = np.loadtxt(
    'doublecouplings_{0:.2f}_{1:.2f}_{2:.2f}_{3:.2f}_{4}.dat'.format(
        c_min[0], c_min[1], c_max[0], c_max[1], npoints))
all_cross_sections = np.loadtxt(
    'doublecrosssection_{0:.2f}_{1:.2f}_{2:.2f}_{3:.2f}_{4}.dat'.format(
        c_min[0], c_min[1], c_max[0], c_max[1], npoints))

In [94]:
# Define Mixture Models
base_mixtures = []
couplings = []
cross_sections = []
for b, indexes in enumerate(all_indexes):
    couplings.append([theano.shared(0.,name='w_{0}_{1}'.format(b,k)) for k in range(len(indexes))])
    cross_sections.append([theano.shared(0.,name='cx_{0}_{1}'.format(b,k)) for k in range(len(indexes))])
    base_mixtures.append(Mixture(indexes, couplings[b], cross_sections[b]))

In [95]:
# For denominator using just the first component
den_indexes = [data_F0]
den_couplings = [theano.shared(1.,name='w_den')]
den_cross_sections = [theano.shared(1., name='cx_den')]
den_mixture = Mixture(den_indexes, den_couplings, den_cross_sections)

In [96]:
csarray_0 = np.linspace(c_min[0], c_max[0], npoints)
csarray_1 = np.linspace(c_min[1], c_max[1], npoints)
decomposedLikelihood = np.zeros((npoints, npoints))
n_effs = np.zeros((npoints, npoints))
alphas = np.zeros((2, npoints, npoints))
ratiosList = []

In [97]:
from carl.ratios import ClassifierRatio
from carl.ratios import DecomposedRatio
from carl.learning import CalibratedClassifierCV

from carl.ratios import ClassifierRatio
from carl.learning import CalibratedClassifierCV
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import StratifiedShuffleSplit

def make_ratio(num, den):
    clf = ExtraTreesClassifier(n_estimators=100, min_samples_split=20, random_state=0, n_jobs=-1)
    cv =  StratifiedShuffleSplit(n_iter=3, test_size=0.5, random_state=1)

    ratio = DecomposedRatio(ClassifierRatio(
        base_estimator=CalibratedClassifierCV(clf, cv=cv, bins=25),
        random_state=0))
    ratio.fit(numerator=num, denominator=den, n_samples=n_samples)
    
    return ratio

In [98]:
base_ratios = []
base_ratios.append(make_ratio(base_mixtures[0],den_mixture))
base_ratios.append(make_ratio(base_mixtures[1],den_mixture))

In [99]:
def get_weights(w):
    n_tot = np.abs(w).sum()
    n_eff = w.sum()
    n_eff = n_eff / n_tot
    alpha = np.exp(-np.sqrt(1./n_eff))
    #alpha1 = np.exp(-neff1**(1./3.))
    return alpha, n_eff

In [101]:
import pdb
target = [1.,0.,0.]
for i, cs_0 in enumerate(csarray_0):
    for j, cs_1 in enumerate(csarray_1):
        target[1] = cs_0
        target[2] = cs_1
        print '{0} {1}'.format(i, j)
        #print target

        # Compute F1 couplings and cross sections
        cs_b0 = all_couplings[i * npoints + j]
        cross_section_b0 = all_cross_sections[i * npoints + j]  
        map(lambda (var, val): var.set_value(val), zip(couplings[0],cs_b0))
        map(lambda (var, val): var.set_value(val), zip(cross_sections[0],cross_section_b0))

        cs_b1 = all_couplings[npoints * npoints + i * npoints + j]
        cross_section_b1 = all_cross_sections[npoints * npoints + i * npoints + j]
        map(lambda (var, val): var.set_value(val), zip(couplings[1],cs_b1))
        map(lambda (var, val): var.set_value(val), zip(cross_sections[1],cross_section_b1))
        # Compute ratios
        ratio_0 = base_ratios[0]
        ratio_1 = base_ratios[1]
        # Compute weights for each ratio
        w_0 = np.multiply(cs_b0, cross_section_b0)
        
        w_1 = np.multiply(cs_b1, cross_section_b1)
        alpha_0, n_eff_0 = get_weights(w_0)
        alpha_1, n_eff_1 = get_weights(w_1)

        alphas[0,i,j] = alpha_0 / (alpha_0 + alpha_1)
        alphas[1,i,j] = alpha_1 / (alpha_0 + alpha_1)
        
        n_effs[i,j] = alphas[0,i,j]*n_eff_0 + alphas[1,i,j]*n_eff_1
        #print(n_effs[i,j])
         
        # Check ratios for negative of non defined values
        r_0 = ratio_0.predict(X_observed)
        r_1 = ratio_1.predict(X_observed)
        z_0 = r_0[np.isinf(r_0)].shape[0]
        z_1 = r_1[np.isinf(r_1)].shape[0]
        #print(z_0,z_1)
        r_0[np.isinf(r_0)] = 0. # Not sure about this
        r_1[np.isinf(r_1)] = 0.
        
        # ratio weighting
        ratio = alphas[0,i,j] * r_0 + alphas[1,i,j] * r_1
        print(z_0,z_1,ratio[ratio <= 0.].shape)
        ratio = ratio[ratio > 0.]
        decomposedLikelihood[i,j] = -np.log(ratio).sum()
        #print(decomposedLikelihood[i,j])

0 0
(1, 0, (96,))
0 1
(1, 0, (98,))
0 2
(1, 0, (112,))
0 3
(1, 0, (76,))
0 4
(1, 0, (32,))
0 5
(1, 0, (5,))
0 6
(1, 0, (1,))
0 7
(1, 0, (14,))
0 8
(1, 0, (18,))
0 9
(1, 0, (18,))
0 10
(1, 0, (15,))
0 11
(1, 0, (11,))
0 12
(1, 0, (54,))
0 13
(1, 0, (88,))
0 14
(1, 0, (79,))
1 0
(1, 0, (105,))
1 1
(1, 0, (46,))
1 2
(1, 0, (47,))
1 3
(1, 0, (33,))
1 4
(1, 0, (22,))
1 5
(1, 0, (2,))
1 6
(1, 0, (0,))
1 7
(1, 0, (4,))
1 8
(1, 0, (6,))
1 9
(1, 0, (0,))
1 10
(1, 0, (0,))
1 11
(1, 0, (16,))
1 12
(1, 0, (68,))
1 13
(1, 0, (83,))
1 14
(1, 0, (68,))
2 0
(1, 0, (96,))
2 1
(1, 0, (6,))
2 2
(1, 0, (0,))
2 3
(1, 0, (1,))
2 4
(1, 0, (0,))
2 5
(1, 0, (0,))
2 6
(1, 0, (0,))
2 7
(1, 0, (0,))
2 8
(1, 0, (0,))
2 9
(1, 0, (0,))
2 10
(1, 0, (2,))
2 11
(1, 0, (15,))
2 12
(1, 0, (18,))
2 13
(1, 0, (28,))
2 14
(1, 0, (20,))
3 0
(1, 0, (28,))
3 1
(1, 0, (0,))
3 2
(1, 0, (0,))
3 3
(1, 0, (0,))
3 4
(1, 0, (0,))
3 5
(1, 0, (0,))
3 6
(1, 0, (0,))
3 7
(1, 0, (0,))
3 8
(1, 0, (0,))
3 9
(1, 0, (0,))
3 10
(1, 0, (2,))
3 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.mlab import griddata
plt.rcParams["figure.figsize"] = (12, 10)
from itertools import product

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel as C, Matern

theta = np.array([x for x in product(csarray_0, csarray_1)])
llr = decomposedLikelihood.flatten()
mle = np.argmin(llr)
llr -= llr[mle]
llr *= 2.
gp = GaussianProcessRegressor(alpha=0.0, normalize_y=True, 
                              kernel=C(1.0) * Matern(1.0, length_scale_bounds="fixed"))
gp.fit(theta, llr)

xi = np.linspace(-0.6, 0.6, 500)
yi = np.linspace(-0.6, 0.6, 500)
    
xx, yy = np.meshgrid(xi, yi)
zz, std = gp.predict(np.c_[xx.ravel(), yy.ravel()], return_std=True)
zi = zz.reshape(xx.shape)

cs = plt.contour(xi, yi, zi, 30, linewidths=0.5, colors='w')
cs = plt.contourf(xi, yi, zi, 30, cmap="viridis",
                  vmax=abs(zi).max(), vmin=0.0)
plt.colorbar()  
#plt.scatter(theta[:, 0], theta[:, 1], marker='o', c='b', s=50, lw=0, zorder=10)
plt.scatter([theta[mle, 0]], [theta[mle, 1]], marker='o', c='r', s=50, lw=0, zorder=10)
plt.scatter([F1[1]], [F1[2]], marker='o', c='w', s=50, lw=0, zorder=10)
plt.xlim(-0.6, 0.6)
plt.ylim(-0.6, 0.6)
plt.xlabel("KHzz")
plt.ylabel("kAzz")
plt.show()

In [18]:
theta = np.array([x for x in product(csarray_0, csarray_1)])
llr = n_effs.flatten()
gp = GaussianProcessRegressor(alpha=0.0, normalize_y=True, 
                              kernel=C(1.0) * Matern(1.0, length_scale_bounds="fixed"))
gp.fit(theta, llr)

xi = np.linspace(-0.6, 0.6, 500)
yi = np.linspace(-0.6, 0.6, 500)
    
xx, yy = np.meshgrid(xi, yi)
zz, std = gp.predict(np.c_[xx.ravel(), yy.ravel()], return_std=True)
zi = zz.reshape(xx.shape)

cs = plt.contour(xi, yi, zi, 30, linewidths=0.5, colors='w')
cs = plt.contourf(xi, yi, zi, 30, cmap="viridis",
                  vmax=abs(zi).max(), vmin=0.0)
plt.colorbar()  
#plt.scatter(theta[:, 0], theta[:, 1], marker='o', c='b', s=50, lw=0, zorder=10)
plt.scatter([F1[1]], [F1[2]], marker='o', c='w', s=50, lw=0, zorder=10)
plt.xlim(-0.6, 0.6)
plt.ylim(-0.6, 0.6)
plt.xlabel("KHzz")
plt.ylabel("kAzz")
plt.show()

ValueError: Found arrays with inconsistent numbers of samples: [29 30]